In [8]:
# -*- coding:utf-8 -*-
# @Python Version: 3.7
# @Time: 2020/5/18 23:31
# @Author: Michael Ming
# @Website: https://michael.blog.csdn.net/
# @File: ks_baselineModel.ipynb
# @Reference: https://www.kaggle.com/learn/feature-engineering
import pandas as pd

ks = pd.read_csv('ks-projects-201801.csv',parse_dates=['deadline','launched'])
ks.head(10)
pd.unique(ks.state)
ks.groupby('state')['ID'].count()

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
Name: ID, dtype: int64